### =====================================================================
### IMPORTACIÓN GENERAL DE LA INFORMACIÓN.
### =====================================================================

In [369]:
## IMPORTACIÓN GENERAL DE LIBRERIAS Y VISUALIZACIÓN DE DATOS (matplotlib y seaborn)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as DT
import warnings
import descartes
import geopandas as gpd
import json
import requests
import geocoder

from sklearn.ensemble import RandomForestRegressor
from shapely.geometry import Point, Polygon
from urllib.request import urlopen #Python3

%matplotlib inline
warnings.filterwarnings('ignore')
plt.style.use('default') 
sns.set(style="whitegrid") 
plt.rcParams['figure.figsize'] = (15, 10)

In [370]:
#lEEMOS EL CSV
train = pd.read_csv('../../DATA_TP2/train.csv')
test = pd.read_csv('../../DATA_TP2/test.csv')
respuesta = pd.read_csv('../../DATA_TP2/ejemploRespuesta.csv')

### =====================================================================
### PREPARACIÓN DE DATOS, FEATURE ENGINEERING.
### =====================================================================

### =====================================================================
### TRATAMIENTO SOBRE TEST.
### =====================================================================

In [371]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 23 columns):
id                            240000 non-null int64
titulo                        234613 non-null object
descripcion                   238381 non-null object
tipodepropiedad               239954 non-null object
direccion                     186928 non-null object
ciudad                        239628 non-null object
provincia                     239845 non-null object
antiguedad                    196445 non-null float64
habitaciones                  217529 non-null float64
garages                       202235 non-null float64
banos                         213779 non-null float64
metroscubiertos               222600 non-null float64
metrostotales                 188533 non-null float64
idzona                        211379 non-null float64
lat                           116512 non-null float64
lng                           116512 non-null float64
fecha                         240

In [372]:
#Borramos las columnas que no nos interesan
train.drop('idzona', axis=1, inplace=True)
train.drop('direccion', axis=1, inplace=True)
train.drop('lat', axis=1, inplace=True)
train.drop('lng', axis=1, inplace=True)

In [373]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 19 columns):
id                            240000 non-null int64
titulo                        234613 non-null object
descripcion                   238381 non-null object
tipodepropiedad               239954 non-null object
ciudad                        239628 non-null object
provincia                     239845 non-null object
antiguedad                    196445 non-null float64
habitaciones                  217529 non-null float64
garages                       202235 non-null float64
banos                         213779 non-null float64
metroscubiertos               222600 non-null float64
metrostotales                 188533 non-null float64
fecha                         240000 non-null object
gimnasio                      240000 non-null float64
usosmultiples                 240000 non-null float64
piscina                       240000 non-null float64
escuelascercanas              240

In [374]:
#Vemos cuales tienen NANs
train.isna().sum()

id                                0
titulo                         5387
descripcion                    1619
tipodepropiedad                  46
ciudad                          372
provincia                       155
antiguedad                    43555
habitaciones                  22471
garages                       37765
banos                         26221
metroscubiertos               17400
metrostotales                 51467
fecha                             0
gimnasio                          0
usosmultiples                     0
piscina                           0
escuelascercanas                  0
centroscomercialescercanos        0
precio                            0
dtype: int64

In [375]:
#Borramos los nulls de las columnas categoricas que no nos interesan
train.dropna(subset=['tipodepropiedad', 'provincia', 'ciudad'], inplace=True)

#Consideramos que las propiedades con garages NAN no poseen y banos poseen 1
train['garages'].fillna(0, inplace=True)
train['banos'].fillna(1, inplace=True)

In [376]:
#Compruebo que no hayan quedado NAN en garages y banos
train.isnull().sum()

id                                0
titulo                         5382
descripcion                    1575
tipodepropiedad                   0
ciudad                            0
provincia                         0
antiguedad                    43328
habitaciones                  22415
garages                           0
banos                             0
metroscubiertos               17341
metrostotales                 51361
fecha                             0
gimnasio                          0
usosmultiples                     0
piscina                           0
escuelascercanas                  0
centroscomercialescercanos        0
precio                            0
dtype: int64

In [377]:
# TRATAMIENTO NAN SOBRE ANTIGUEDAD, HABITACIONES, METROSCUBIERTOS y METROSTOTALES
# Hice el promedio por ciudad y las que seguian estando en NAN
# ya sea porque en esa ciudad todas las propiedades estaban en NAN
# o talvez habia solo 1 propiedad con NAN
# le hice el mean general considerando toda esa columna
# Antiguedad = 0 -> propiedad a estrenar
# No hay propiedades con habitaciones, metroscubiertos o metrostotales en 0.

# Antiguedad
promedio_antiguedad_x_ciudad = train.groupby('ciudad')['antiguedad'].transform('mean')
train['antiguedad'].fillna(promedio_antiguedad_x_ciudad, inplace=True)
train['antiguedad'].fillna(train['antiguedad'].mean(), inplace=True)

# Habitaciones
promedio_habitaciones_x_ciudad = train.groupby('ciudad')['habitaciones'].transform('mean')
train['habitaciones'].fillna(promedio_habitaciones_x_ciudad, inplace=True)
train['habitaciones'].fillna(train['habitaciones'].mean(), inplace=True)

#Metros cubiertos
promedio_metroscubierto_x_ciudad = train.groupby('ciudad')['metroscubiertos'].transform('mean')
train['metroscubiertos'].fillna(promedio_metroscubierto_x_ciudad, inplace=True)
train['metroscubiertos'].fillna(train['metroscubiertos'].mean(), inplace=True)

#Metros totales
promedio_metrostotales_x_ciudad = train.groupby('ciudad')['metrostotales'].transform('mean')
train['metrostotales'].fillna(promedio_metrostotales_x_ciudad, inplace=True)
train['metrostotales'].fillna(train['metrostotales'].mean(), inplace=True)

#convierto los valores a int para no tener decimales
train['antiguedad'].astype(int)
train['habitaciones'].astype(int)
train['metroscubiertos'].astype(int)
train['metrostotales'].astype(int)

#Compruebo que no haya quedado ni un NAN
train.isnull().sum()

id                               0
titulo                        5382
descripcion                   1575
tipodepropiedad                  0
ciudad                           0
provincia                        0
antiguedad                       0
habitaciones                     0
garages                          0
banos                            0
metroscubiertos                  0
metrostotales                    0
fecha                            0
gimnasio                         0
usosmultiples                    0
piscina                          0
escuelascercanas                 0
centroscomercialescercanos       0
precio                           0
dtype: int64

# NOTA :
# FALTA VER TITULO Y DESCRIPCION

### =====================================================================
### TRATAMIENTO SOBRE TEST.
### =====================================================================

In [378]:
test.shape

(60000, 22)

In [379]:
#Borramos las columnas que no nos interesan
test.drop('idzona', axis=1, inplace=True)
test.drop('direccion', axis=1, inplace=True)
test.drop('lat', axis=1, inplace=True)
test.drop('lng', axis=1, inplace=True)

In [380]:
#Vemos cuales tienen NANs
test.isna().sum()

id                                0
titulo                         1378
descripcion                     401
tipodepropiedad                   7
ciudad                           83
provincia                        42
antiguedad                    10714
habitaciones                   5628
garages                        9323
banos                          6554
metroscubiertos                4299
metrostotales                 12655
fecha                             0
gimnasio                          0
usosmultiples                     0
piscina                           0
escuelascercanas                  0
centroscomercialescercanos        0
dtype: int64

In [381]:
#Borramos los nulls de las columnas categoricas que no nos interesan
#test.dropna(subset=['tipodepropiedad', 'provincia', 'ciudad'], inplace=True)

#Consideramos que las propiedades con garages NAN no poseen y banos poseen 1
test['garages'].fillna(0, inplace=True)
test['banos'].fillna(1, inplace=True)

In [382]:
#Compruebo que no hayan quedado NULL
test.isna().sum()

id                                0
titulo                         1378
descripcion                     401
tipodepropiedad                   7
ciudad                           83
provincia                        42
antiguedad                    10714
habitaciones                   5628
garages                           0
banos                             0
metroscubiertos                4299
metrostotales                 12655
fecha                             0
gimnasio                          0
usosmultiples                     0
piscina                           0
escuelascercanas                  0
centroscomercialescercanos        0
dtype: int64

In [383]:
# TRATAMIENTO NAN SOBRE ANTIGUEDAD, HABITACIONES, METROSCUBIERTOS y METROSTOTALES
# Hice el promedio por ciudad y las que seguian estando en NAN
# ya sea porque en esa ciudad todas las propiedades estaban en NAN
# o talvez habia solo 1 propiedad con NAN
# le hice el mean general considerando toda esa columna
# Antiguedad = 0 -> propiedad a estrenar
# No hay propiedades con habitaciones, metroscubiertos o metrostotales en 0.

# Antiguedad
promedio_antiguedad_x_ciudad = test.groupby('ciudad')['antiguedad'].transform('mean')
test['antiguedad'].fillna(promedio_antiguedad_x_ciudad, inplace=True)
test['antiguedad'].fillna(test['antiguedad'].mean(), inplace=True)

# Habitaciones
promedio_habitaciones_x_ciudad = test.groupby('ciudad')['habitaciones'].transform('mean')
test['habitaciones'].fillna(promedio_habitaciones_x_ciudad, inplace=True)
test['habitaciones'].fillna(test['habitaciones'].mean(), inplace=True)

#Metros cubiertos
promedio_metroscubierto_x_ciudad = test.groupby('ciudad')['metroscubiertos'].transform('mean')
test['metroscubiertos'].fillna(promedio_metroscubierto_x_ciudad, inplace=True)
test['metroscubiertos'].fillna(test['metroscubiertos'].mean(), inplace=True)

#Metros totales
promedio_metrostotales_x_ciudad = test.groupby('ciudad')['metrostotales'].transform('mean')
test['metrostotales'].fillna(promedio_metrostotales_x_ciudad, inplace=True)
test['metrostotales'].fillna(test['metrostotales'].mean(), inplace=True)

#convierto los valores a int para no tener decimales
test['antiguedad'].astype(int)
test['habitaciones'].astype(int)
test['metroscubiertos'].astype(int)
test['metrostotales'].astype(int)

#Compruebo que no haya quedado ni un NAN
test.isnull().sum()

id                               0
titulo                        1378
descripcion                    401
tipodepropiedad                  7
ciudad                          83
provincia                       42
antiguedad                       0
habitaciones                     0
garages                          0
banos                            0
metroscubiertos                  0
metrostotales                    0
fecha                            0
gimnasio                         0
usosmultiples                    0
piscina                          0
escuelascercanas                 0
centroscomercialescercanos       0
dtype: int64

# NOTA!

# Tenemos que ver que hacemos con los NAN de TIPO DE PROPIEDAD, CIUDAD Y PROVINCIA porque no se pueden droppear
    
# Falta ver que hacemos con titulo y descripcion


In [384]:
test.shape

(60000, 18)

In [54]:
#Para que ande el Random Forest

In [55]:
train['fecha'] = pd.to_datetime(train['fecha'], infer_datetime_format=True)
test['fecha'] = pd.to_datetime(test['fecha'], infer_datetime_format=True)
train['anio'] = train['fecha'].dt.year
test['anio'] = test['fecha'].dt.year
train.drop('fecha', axis=1, inplace=True)
test.drop('fecha', axis=1, inplace=True)


In [56]:
train.to_csv('DATA/train.csv', index=False)
test.to_csv('DATA/test.csv', index=False)

In [57]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 13 columns):
id                            60000 non-null int64
antiguedad                    60000 non-null float64
habitaciones                  60000 non-null float64
garages                       60000 non-null float64
banos                         60000 non-null float64
metroscubiertos               60000 non-null float64
metrostotales                 60000 non-null float64
gimnasio                      60000 non-null float64
usosmultiples                 60000 non-null float64
piscina                       60000 non-null float64
escuelascercanas              60000 non-null float64
centroscomercialescercanos    60000 non-null float64
anio                          60000 non-null int64
dtypes: float64(11), int64(2)
memory usage: 6.0 MB
